# Analisis para saber si se van a alinear o se pueden alinear adecuadamente los candidatos

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('../')
sys.path.append('../math_tricks/')
import funciones_CLICK as fc
from scipy.spatial.distance import pdist, squareform
import multiprocessing
from functools import partial
import warnings
warnings.simplefilter("ignore")

%matplotlib inline

--------------------------------------------
__Encontramos que cuando se rota y traslada [referencia](https://academic.oup.com/nar/article/45/1/e5/2871113), hay que calcular la distancia de UN residuo $A_i$ VS todos los $B_j$ y quedarnos con la pareja $(A_i,B_j)$ tal que su distancia sea menor a 3.5 A y si hay mas de uno te quedas con el menor!!__

## Encontrar matcheo de residuo a residuo, en lo que calcula la rotacion y traslacion.

In [3]:
new_df_cliques1 = pd.read_pickle('../clique1.pkl')
new_df_cliques2 = pd.read_pickle('../clique2.pkl')

candidatos = pd.read_pickle('../parejas.pkl')

df_atoms1 = pd.read_pickle('../clique1_df_atributos.pkl')
df_atoms2 = pd.read_pickle('../clique2_df_atributos.pkl')

rmsd = pd.read_pickle("../df_alineados.pkl").reset_index(drop=True)

In [4]:
pd.DataFrame(candidatos) # RESIDUOS

,0,1
0,98.0000,160.0000
1,99.0000,161.0000
2,100.0000,121.0000
3,86.0000,156.0000
4,87.0000,157.0000
5,88.0000,158.0000
6,89.0000,159.0000


In [9]:
df_atoms2.iloc[795]

atom_number       796                                                                                                       
atom_name         CA                                                                                                        
residue_name      PHE                                                                                                       
residue_number    162                                                                                                       
vector            [5.852, 1.707, 14.766]                                                                                    
ngb               [87, 88, 89, 90, 117, 118, 119, 120, 121, 133, 134, 137, 152, 153, 154, 155, 156, 157, 160, 161, 163, 164]
Name: 795, dtype: object

In [10]:
rmsd.sort_values('distancia',ascending=True).head()

,distancia,candigatos,cand_1,cand2
0,0.3368,"(484, 795)",484,795
1,0.9679,"(250, 709)",250,709
2,1.0505,"(341, 299)",341,299
3,1.1205,"(196, 833)",196,833
4,1.3781,"(120, 242)",120,242


In [11]:
rmsd

,distancia,candigatos,cand_1,cand2
0,0.3368,"(484, 795)",484,795
1,0.9679,"(250, 709)",250,709
2,1.0505,"(341, 299)",341,299
3,1.1205,"(196, 833)",196,833
4,1.3781,"(120, 242)",120,242
5,1.4080,"(179, 846)",179,846
6,1.5792,"(475, 376)",475,376
7,1.6667,"(26, 1)",26,1
8,1.6835,"(382, 604)",382,604
9,1.7145,"(305, 621)",305,621


In [12]:
%%time
a_1 = df_atoms1[df_atoms1.atom_name == 'CA'].vector.values

lista_vectores_gorro = []
for bari in new_df_cliques1.baricentro_clique.values:
    lista_pre_vectores = []

    for coord in a_1:
        lista_pre_vectores.append(coord - bari)

    lista_vectores_gorro.append(lista_pre_vectores)

CPU times: user 24.1 ms, sys: 3.64 ms, total: 27.7 ms
Wall time: 33.2 ms


In [13]:
vectores_gorro_proteina_1 = pd.DataFrame(lista_vectores_gorro)

In [14]:
%%time
a_2 = df_atoms2[df_atoms2.atom_name == 'CA'].vector.values

lista_vectores_gorro = []
for bari in new_df_cliques2.baricentro_clique.values:
    lista_pre_vectores = []
    for coord in a_2:
        lista_pre_vectores.append(coord - bari)

    lista_vectores_gorro.append(lista_pre_vectores)

CPU times: user 30.4 ms, sys: 0 ns, total: 30.4 ms
Wall time: 34.1 ms


In [15]:
vectores_gorro_proteina_2 = pd.DataFrame(lista_vectores_gorro)

In [16]:
vectores_gorro_proteina_1.shape

(276, 71)

In [17]:
for i,j in enumerate(rmsd.columns):
    print(i,j)

0 distancia
1 candigatos
2 cand_1
3 cand2


In [18]:
for i,j in enumerate(new_df_cliques1.columns):
    print(i,j)

0 0
1 1
2 2
3 3
4 4
5 5
6 6
7 ss_0
8 ss_1
9 ss_2
10 ss_3
11 ss_4
12 ss_5
13 ss_6
14 coord_clique_0
15 coord_clique_1
16 coord_clique_2
17 coord_clique_3
18 coord_clique_4
19 coord_clique_5
20 coord_clique_6
21 matriz_coordenadas
22 baricentro_clique
23 vec_gorro_0
24 vec_gorro_1
25 vec_gorro_2
26 vec_gorro_3
27 vec_gorro_4
28 vec_gorro_5
29 vec_gorro_6
30 vectores_gorro
31 distancia_promedio


In [20]:
# idx = id_so = 354
so = 0.0
idx = 0
df_rmsd = rmsd.values
df_new_df_clique1 = new_df_cliques1.values
df_new_df_clique2 = new_df_cliques2.values
df_vectores_gorro_proteina_1 = vectores_gorro_proteina_1.values
df_vectores_gorro_proteina_2 = vectores_gorro_proteina_2.values

atoms1 = df_atoms1.atom_number.values
atoms2 = df_atoms2.atom_number.values
number_of_residues = df_atoms1.shape[0]

protein_to_compare = np.array([i for i in df_atoms2.vector.values])

if so == 1.0:
    id_so = idx
    print(so, id_so)
    print('////' * 20)


clique_in_protein_1 = df_rmsd[idx, 3]  # se extrae la info de los candidatos
clique_in_protein_2 = df_rmsd[idx, 4]

# se obtienen el numero de residuo de los candidatos
los_del_clique_1 = df_new_df_clique1[clique_in_protein_1, [0, 1, 2, 3, 4, 5, 6]]
los_del_clique_2 = df_new_df_clique1[clique_in_protein_2, [0, 1, 2, 3, 4, 5, 6]]

matriz_rotacion = df_rmsd[idx, 2]  # se obtiene la matriz de rotacion de los cliques candidatos

# se obtienen los vectores gorro de la proteina a rotar y trasladar
vector_gorro = df_vectores_gorro_proteina_1[clique_in_protein_1]
# se aplica la rotacion a vectores gorro
coord_vectores_rotados = [np.matmul(matriz_rotacion, i.reshape(3, 1)).T[0] for i in vector_gorro]
# se obtiene el baricentro de la proteina 2
baricentro_proteina_2 = df_new_df_clique2[clique_in_protein_2, 22]
# se suma el baricentro a vectores rotados de la proteina 1
protein_trasladado_rotado = coord_vectores_rotados + baricentro_proteina_2  # nuevas coordendas proteina 1
# se quitan residuos que pertenezcan al clique candidato para calcular la distancia y emparejar al mejor.
protein_trasladado_rotado_sin_clique = np.delete(protein_trasladado_rotado, los_del_clique_1, 0)
protein_to_compare_sin_clique = np.delete(protein_to_compare, los_del_clique_2, 0)
# numeros de atomos
atom_numbers1 = np.delete(atoms1, los_del_clique_1, 0)
atom_numbers2 = np.delete(atoms2, los_del_clique_2, 0)

residuos_match = []  # aqui se guardan las parejas de residuos

# se itera por cada residuo ya rotado y trasladado
for residuo_not_in_clique, res_1 in zip(protein_trasladado_rotado_sin_clique, atom_numbers1):
          for residuo, res_2 in zip(protein_to_compare_sin_clique, atom_numbers2):
            distancia = pdist(np.array([residuo_not_in_clique, residuo]), metric='euclidean').item()
            if distancia < 3.5:
                continue
#                 list_cand =[pdist(np.array([residuo_not_in_clique, res_ngb.coord]), metric='euclidean').item()
#                      for res_ngb in residuo.ngb]
#                 break
#                 residuos_match.append([distancia, (res_1, res_2)])

df = pd.DataFrame(residuos_match, columns=['distancia', 'candigatos'])


IndexError: index 4 is out of bounds for axis 1 with size 4

In [21]:
id_so = 0
so = 0.0

def align_residues(idx, so):

    df_rmsd = rmsd.values
    df_new_df_clique1 = new_df_cliques1.values
    df_new_df_clique2 = new_df_cliques2.values
    df_vectores_gorro_proteina_1 = vectores_gorro_proteina_1.values
    df_vectores_gorro_proteina_2 = vectores_gorro_proteina_2.values

    atoms1 = df_atoms1[df_atoms1.atom_name == 'CA'].atom_number.values
    atoms2 = df_atoms2[df_atoms1.atom_name == 'CA'].atom_number.values
    number_of_residues = df_atoms1[df_atoms1.atom_name == 'CA'].shape[0]
    
    protein_to_compare = np.array([i for i in df_atoms2[df_atoms2.atom_name == 'CA'].vector.values])
    
    clique_in_protein_1 = df_rmsd[idx, 3]  # se extrae la info de los candidatos
    clique_in_protein_2 = df_rmsd[idx, 4]

    # se obtienen el numero de residuo de los candidatos
    los_del_clique_1 = df_new_df_clique1[clique_in_protein_1, [0, 1, 2, 3, 4, 5, 6]]
    los_del_clique_2 = df_new_df_clique1[clique_in_protein_2, [0, 1, 2, 3, 4, 5, 6]]

    matriz_rotacion = df_rmsd[idx, 2]  # se obtiene la matriz de rotacion de los cliques candidatos

    # se obtienen los vectores gorro de la proteina a rotar y trasladar
    vector_gorro = df_vectores_gorro_proteina_1[clique_in_protein_1]
    # se aplica la rotacion a vectores gorro
    coord_vectores_rotados = [np.matmul(matriz_rotacion, i.reshape(3, 1)).T[0] for i in vector_gorro]
    # se obtiene el baricentro de la proteina 2
    baricentro_proteina_2 = df_new_df_clique2[clique_in_protein_2, 22]
    # se suma el baricentro a vectores rotados de la proteina 1
    protein_trasladado_rotado = coord_vectores_rotados + baricentro_proteina_2  # nuevas coordendas proteina 1
    # se quitan residuos que pertenezcan al clique candidato para calcular la distancia y emparejar al mejor.
    protein_trasladado_rotado_sin_clique = np.delete(protein_trasladado_rotado, los_del_clique_1, 0)
    protein_to_compare_sin_clique = np.delete(protein_to_compare, los_del_clique_2, 0)
    # numeros de atomos
    atom_numbers1 = np.delete(atoms1, los_del_clique_1, 0)
    atom_numbers2 = np.delete(atoms2, los_del_clique_2, 0)

    residuos_match = []  # aqui se guardan las parejas de residuos

    # se itera por cada residuo ya rotado y trasladado
 
    for residuo_not_in_clique, res_1 in zip(protein_trasladado_rotado_sin_clique, atom_numbers1):
        for residuo, res_2 in zip(protein_to_compare_sin_clique, atom_numbers2):
            distancia = pdist(np.array([residuo_not_in_clique, residuo]), metric='euclidean').item()
            if distancia < 3.5:
                residuos_match.append([distancia, (res_1, res_2)])

    df = pd.DataFrame(residuos_match, columns=['distancia', 'candigatos'])
    df = df.sort_values('distancia')
    df['cand1'] = df.candigatos.str[0]
    df['cand2'] = df.candigatos.str[1]

    c1 = []
    c2 = []
    cand_n = []
    for i in df.values:

        if (i[2] in c1) or (i[3] in c2) or (i[0] > 3.5):
            continue
        else:
            c1.append(i[2])
            c2.append(i[3])
            cand_n.append(i)

    so_temp = len(cand_n) / number_of_residues

    return idx, so_temp

In [22]:
id_so = 0
so = 0.0
for i in np.arange(len(df_rmsd))[:400]:
    id_so, so_temp = (align_residues(i,so))
    if so <= so_temp:
        so = so_temp
        id_so = i
        print(id_so,so)
    if so == 1:
        break

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match

In [23]:
%%time
align_residues(143,0)

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match

In [24]:
((0.80 * 33740)/60)/60

7.497777777777778

In [25]:
((0.80 * 500)/60)

6.666666666666667

# check cliques seguimiento

In [54]:
c1_7 = pd.read_pickle('../cliques1_7.pkl')
c1_7.head()

,0,1,2,3,4,5,6
38,1,10,12,9,6,7,8
39,1,10,12,6,9,7,8
40,1,10,12,6,7,8,9
41,1,10,12,6,7,9,8
77,5,6,7,8,1,9,12


In [55]:
c1_6 = pd.read_pickle('../cliques1_6.pkl')
c1_6[c1_6[0]==1][c1_6[1]==10]

,0,1,2,3,4,5
49,1,10,12,9,6,8
50,1,10,12,9,6,7
51,1,10,12,6,9,8
52,1,10,12,6,9,7
53,1,10,12,6,7,8
54,1,10,12,6,7,9


In [56]:
c1_5 = pd.read_pickle('../cliques1_5.pkl')
c1_5[c1_5[0]==1][c1_5[1]==10]

,0,1,2,3,4
32,1,10,12,9,8
33,1,10,12,9,6
34,1,10,12,9,7
35,1,10,12,6,8
36,1,10,12,6,9
37,1,10,12,6,7


In [57]:
c1_4 = pd.read_pickle('../cliques1_4.pkl')
c1_4[c1_4[0]==1][c1_4[1]==10]

,0,1,2,3
29,1,10,12,8
30,1,10,12,9
31,1,10,12,6
32,1,10,12,7


In [58]:
c1_3 = pd.read_pickle('../cliques1_3.pkl')
c1_3[c1_3[0]==1][c1_3[1]==10]

,0,1,2
4,1,10,6
18,1,10,7
24,1,10,12


In [60]:
c1_3.iloc[1]

0    1
1    2
2    4
Name: 171, dtype: int64

In [66]:
c1_3.sort_index().iloc[1]

0    8 
1    10
2    6 
Name: 1, dtype: int64

In [67]:
c2_3 = pd.read_pickle('../cliques2_3.pkl')
c2_3.sort_index().iloc[78]

0    8
1    2
2    6
Name: 78, dtype: int64

In [62]:
 pd.read_pickle('clique1.pkl')

,0,1,2,3,4,5,6,ss_0,ss_1,ss_2,ss_3,ss_4,ss_5,ss_6,coord_clique_0,coord_clique_1,coord_clique_2,coord_clique_3,coord_clique_4,coord_clique_5,coord_clique_6,matriz_coordenadas,baricentro_clique,vec_gorro_0,vec_gorro_1,vec_gorro_2,vec_gorro_3,vec_gorro_4,vec_gorro_5,vec_gorro_6,vectores_gorro,distancia_promedio
0,64.0000,128.0000,63.0000,127.0000,61.0000,62.0000,31.0000,B,C,B,B,C,C,C,"[47.908, 20.394, 35.638]","[40.512, 27.82, 44.23]","[48.73, 19.768, 35.292]","[41.218, 28.83, 42.508]","[47.136, 18.402, 34.988]","[48.75, 17.752, 34.564]","[50.744, 14.238, 31.226]","[[47.908, 20.394, 35.638], [40.512, 27.82, 44.23], [48.73, 19.768, 35.292], [41.218, 28.83, 42.508], [47.136, 18.402, 34.988], [48.75, 17.752, 34.564], [50.744, 14.238, 31.226]]","[46.42828571428572, 21.02914285714286, 36.920857142857145]","[1.4797142857142802, -0.6351428571428599, -1.2828571428571465]","[-5.916285714285721, 6.790857142857142, 7.309142857142852]","[2.301714285714276, -1.2611428571428576, -1.628857142857143]","[-5.2102857142857175, 7.80085714285714, 5.587142857142858]","[0.7077142857142817, -2.6271428571428572, -1.932857142857145]","[2.321714285714279, -3.2771428571428594, -2.3568571428571445]","[4.315714285714279, -6.791142857142859, -5.6948571428571455]","[[1.4797142857142802, -0.6351428571428599, -1.2828571428571465], [-5.916285714285721, 6.790857142857142, 7.309142857142852], [2.301714285714276, -1.2611428571428576, -1.628857142857143], [-5.2102857142857175, 7.80085714285714, 5.587142857142858], [0.7077142857142817, -2.6271428571428572, -1.932857142857145], [2.321714285714279, -3.2771428571428594, -2.3568571428571445], [4.315714285714279, -6.791142857142859, -5.6948571428571455]]",6.5025
1,133.0000,134.0000,135.0000,31.0000,125.0000,126.0000,127.0000,C,C,B,C,B,B,B,"[40.698, 30.678, 47.526]","[41.506, 30.482, 48.232]","[40.034, 29.818, 47.616]","[50.744, 14.238, 31.226]","[41.01, 29.0, 43.564]","[40.32, 29.844, 43.574]","[41.218, 28.83, 42.508]","[[40.698, 30.678, 47.526], [41.506, 30.482, 48.232], [40.034, 29.818, 47.616], [50.744, 14.238, 31.226], [41.01, 29.0, 43.564], [40.32, 29.844, 43.574], [41.218, 28.83, 42.508]]","[42.21857142857142, 27.555714285714284, 43.46371428571428]","[-1.5205714285714222, 3.1222857142857166, 4.062285714285721]","[-0.7125714285714224, 2.926285714285715, 4.768285714285717]","[-2.1845714285714237, 2.262285714285717, 4.152285714285718]","[8.525428571428577, -13.317714285714285, -12.237714285714283]","[-1.2085714285714246, 1.4442857142857157, 0.10028571428571809]","[-1.8985714285714224, 2.288285714285717, 0.1102857142857161]","[-1.0005714285714191, 1.274285714285714, -0.9557142857142793]","[[-1.5205714285714222, 3.1222857142857166, 4.062285714285721], [-0.7125714285714224, 2.926285714285715, 4.768285714285717], [-2.1845714285714237, 2.262285714285717, 4.152285714285718], [8.525428571428577, -13.317714285714285, -12.237714285714283], [-1.2085714285714246, 1.4442857142857157, 0.10028571428571809], [-1.8985714285714224, 2.288285714285717, 0.1102857142857161], [-1.0005714285714191, 1.274285714285714, -0.9557142857142793]]",6.1329
2,66.0000,134.0000,135.0000,136.0000,124.0000,125.0000,126.0000,B,C,B,B,B,B,B,"[49.822, 19.636, 36.366]","[41.506, 30.482, 48.232]","[40.034, 29.818, 47.616]","[40.148, 32.094, 47.688]","[42.846, 30.28, 43.85]","[41.01, 29.0, 43.564]","[40.32, 29.844, 43.574]","[[49.822, 19.636, 36.366], [41.506, 30.482, 48.232], [40.034, 29.818, 47.616], [40.148, 32.094, 47.688], [42.846, 30.28, 43.85], [41.01, 29.0, 43.564], [40.32, 29.844, 43.574]]","[42.24085714285714, 28.736285714285714, 44.41285714285714]","[7.581142857142865, -9.100285714285715, -8.046857142857142]","[-0.7348571428571375, 1.7457142857142856, 3.8191428571428574]","[-2.206857142857139, 1.0817142857142876, 3.2031428571428577]","[-2.0928571428571345, 3.3577142857142874, 3.2751428571428605]","[0.6051428571428588, 1.5437142857142874, -0.5628571428571405]","[-1.2308571428571398, 0.26371428571428623, -0.8488571428571419]","[-1.9208571428571375, 1.10771